In [1]:
from bs4 import BeautifulSoup

import requests
import pandas as pd
import re
import math
import time


In [2]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36',
}

In [3]:
keyword = '인공지능 문제'

In [4]:
url= 'https://news.joins.com/Search/TotalNews?page=%d&Keyword=%s&PeriodType=DirectInput&StartSearchDate=2016.01.01&EndSearchDate=2020.12.31&SortType=New&SearchCategoryType=TotalNews'% (1,keyword)

In [5]:
res = requests.get(url, headers=header)

In [6]:
#### 페이지 수 확인
soup=BeautifulSoup(res.content, 'html.parser')

page_base=soup.find("span",{'class':'total_number'}).text
page_base1=re.split('\/',page_base)[0]

page_end=int(re.split('-', page_base1)[1])

In [11]:
### 기사 링크 수집

href_all=[]
check_list=[]#가져오지 못한 페이지 있는지 확인하기 위해 만든 리스트

for page in range(1,page_end+1) :
    print('page :' + str(page) + '/' + str(page_end))
    
    url= 'https://news.joins.com/Search/TotalNews?page=%d&Keyword=%s&PeriodType=DirectInput&StartSearchDate=2016.01.01&EndSearchDate=2020.12.31&SortType=New&SearchCategoryType=TotalNews'% (page,keyword)
    
    res = requests.get(url, headers=header)  
    
    #데이터를 잘 가져오고 있는지 확인
    
    if res.status_code==200:
        check_list.append('정상')
    else:
        check_list.append('에러')
        continue
        
    ### 2페이지 이상부터 페이지 정보 안가져와졌으면 다시 가져오기
    
    if page>1:
        if soup==BeautifulSoup(res.content, 'html.parser'):
            while True:
                res = requests.get(url, headers=header)
                if soup!=BeautifulSoup(res.content, 'html.parser'):
                    break
    
    
    ### 기사 링크 수집
    
    soup=BeautifulSoup(res.content, 'html.parser')
    
    href = soup.find_all("h2",{'class':'headline mg'})
   
    for i in href:
        href_all.append(i.find('a')['href'])
    
    ### 페이지의 데이터를 전부 가져오지 못했으면 다시 가져오기
    
    if page!=page_end:
        if len(href) !=10:
            while True:
                res = requests.get(url, headers=header)  
                soup==BeautifulSoup(res.content, 'html.parser')
                res = requests.get(url, headers=header)  
                
                soup=BeautifulSoup(res.content, 'html.parser')
    
                href = soup.find_all("h2",{'class':'headline mg'})

                for i in href:
                    href_all.append(i.find('a')['href'])
                    
                if len(href)==10:
                    break
    
    href=[]

page :1/498
page :2/498
page :3/498
page :4/498
page :5/498
page :6/498
page :7/498
page :8/498
page :9/498
page :10/498
page :11/498
page :12/498
page :13/498
page :14/498
page :15/498
page :16/498
page :17/498
page :18/498
page :19/498
page :20/498
page :21/498
page :22/498
page :23/498
page :24/498
page :25/498
page :26/498
page :27/498
page :28/498
page :29/498
page :30/498
page :31/498
page :32/498
page :33/498
page :34/498
page :35/498
page :36/498
page :37/498
page :38/498
page :39/498
page :40/498
page :41/498
page :42/498
page :43/498
page :44/498
page :45/498
page :46/498
page :47/498
page :48/498
page :49/498
page :50/498
page :51/498
page :52/498
page :53/498
page :54/498
page :55/498
page :56/498
page :57/498
page :58/498
page :59/498
page :60/498
page :61/498
page :62/498
page :63/498
page :64/498
page :65/498
page :66/498
page :67/498
page :68/498
page :69/498
page :70/498
page :71/498
page :72/498
page :73/498
page :74/498
page :75/498
page :76/498
page :77/498
page :78

In [12]:
def date_slice(date_base):
    date_base1=re.sub('[^0-9]','',date_base)
    year=date_base1[0:4]
    month = date_base1[4:6]
    day = date_base1[6:8]
    return(year,month,day)

In [ ]:
check_list_2=[]
data_all = pd.DataFrame()
page=0

for url in href_all :
    
    page= page+1
    print('page :' + str(page) + '/' + str(len(href_all)))
    
    res = requests.get(url, headers=header)  
    
    if res.status_code==200:
        check_list_2.append('정상')
    else:
        check_list_2.append('에러')
        continue
        

    if page>1:
        if soup==BeautifulSoup(res.content, 'html.parser'):
            while True:
                res = requests.get(url, headers=header)
                if soup!=BeautifulSoup(res.content, 'html.parser'):
                    break
    
    
    
    soup=BeautifulSoup(res.content, 'html.parser')
    
    title = soup.select_one('div > h1').text
    
    try:
        content=soup.find('div',{'class':'article_body mg fs4'}).text
    except:
        content = soup.find('div',{'class':'article_body fs1 mg'}).text
    
    date_base=soup.find('div',{'class':'byline'}).text
    year=date_slice(date_base)[0]
    month=date_slice(date_base)[1]
    day=date_slice(date_base)[2]
    
    data = {"Title":[title], "Content" : [content], "Year" :[year], "Month" : [month], "Day" : [day]}
    data = pd.DataFrame(data)
    
    data_all = pd.concat([data_all,data], axis=0)
    
data_all=data_all.reset_index().drop(['index'],axis=1)


page :1/4980
page :2/4980
page :3/4980
page :4/4980
page :5/4980
page :6/4980
page :7/4980
page :8/4980
page :9/4980
page :10/4980
page :11/4980
page :12/4980
page :13/4980
page :14/4980
page :15/4980
page :16/4980
page :17/4980
page :18/4980
page :19/4980
page :20/4980
page :21/4980
page :22/4980
page :23/4980
page :24/4980
page :25/4980
page :26/4980
page :27/4980
page :28/4980
page :29/4980
page :30/4980
page :31/4980
page :32/4980
page :33/4980
page :34/4980
page :35/4980
page :36/4980
page :37/4980
page :38/4980
page :39/4980
page :40/4980
page :41/4980
page :42/4980
page :43/4980
page :44/4980
page :45/4980
page :46/4980
page :47/4980
page :48/4980
page :49/4980
page :50/4980
page :51/4980
page :52/4980
page :53/4980
page :54/4980
page :55/4980
page :56/4980
page :57/4980
page :58/4980
page :59/4980
page :60/4980
page :61/4980
page :62/4980
page :63/4980
page :64/4980
page :65/4980
page :66/4980
page :67/4980
page :68/4980
page :69/4980
page :70/4980
page :71/4980
page :72/4980
p

page :555/4980
page :556/4980
page :557/4980
page :558/4980
page :559/4980
page :560/4980
page :561/4980
page :562/4980
page :563/4980
page :564/4980
page :565/4980
page :566/4980
page :567/4980
page :568/4980
page :569/4980
page :570/4980
page :571/4980
page :572/4980
page :573/4980
page :574/4980
page :575/4980
page :576/4980
page :577/4980
page :578/4980
page :579/4980
page :580/4980
page :581/4980
page :582/4980
page :583/4980
page :584/4980
page :585/4980
page :586/4980
page :587/4980
page :588/4980
page :589/4980
page :590/4980
page :591/4980
page :592/4980
page :593/4980
page :594/4980
page :595/4980
page :596/4980
page :597/4980
page :598/4980
page :599/4980
page :600/4980
page :601/4980
page :602/4980
page :603/4980
page :604/4980
page :605/4980
page :606/4980
page :607/4980
page :608/4980
page :609/4980
page :610/4980
page :611/4980
page :612/4980
page :613/4980
page :614/4980
page :615/4980
page :616/4980
page :617/4980
page :618/4980
page :619/4980
page :620/4980
page :621/

page :1095/4980
page :1096/4980
page :1097/4980
page :1098/4980
page :1099/4980
page :1100/4980
page :1101/4980
page :1102/4980
page :1103/4980
page :1104/4980
page :1105/4980
page :1106/4980
page :1107/4980
page :1108/4980
page :1109/4980
page :1110/4980
page :1111/4980
page :1112/4980
page :1113/4980
page :1114/4980
page :1115/4980
page :1116/4980
page :1117/4980
page :1118/4980
page :1119/4980
page :1120/4980
page :1121/4980
page :1122/4980
page :1123/4980
page :1124/4980
page :1125/4980
page :1126/4980
page :1127/4980
page :1128/4980
page :1129/4980
page :1130/4980
page :1131/4980
page :1132/4980
page :1133/4980
page :1134/4980
page :1135/4980
page :1136/4980
page :1137/4980
page :1138/4980
page :1139/4980
page :1140/4980
page :1141/4980
page :1142/4980
page :1143/4980
page :1144/4980
page :1145/4980
page :1146/4980
page :1147/4980
page :1148/4980
page :1149/4980
page :1150/4980
page :1151/4980
page :1152/4980
page :1153/4980
page :1154/4980
page :1155/4980
page :1156/4980
page :11

page :1608/4980
page :1609/4980
page :1610/4980
page :1611/4980
page :1612/4980
page :1613/4980
page :1614/4980
page :1615/4980
page :1616/4980
page :1617/4980
page :1618/4980
page :1619/4980
page :1620/4980
page :1621/4980
page :1622/4980
page :1623/4980
page :1624/4980
page :1625/4980
page :1626/4980
page :1627/4980
page :1628/4980
page :1629/4980
page :1630/4980
page :1631/4980
page :1632/4980
page :1633/4980
page :1634/4980
page :1635/4980
page :1636/4980
page :1637/4980
page :1638/4980
page :1639/4980
page :1640/4980
page :1641/4980
page :1642/4980
page :1643/4980
page :1644/4980
page :1645/4980
page :1646/4980
page :1647/4980
page :1648/4980
page :1649/4980
page :1650/4980
page :1651/4980
page :1652/4980
page :1653/4980
page :1654/4980
page :1655/4980
page :1656/4980
page :1657/4980
page :1658/4980
page :1659/4980
page :1660/4980
page :1661/4980
page :1662/4980
page :1663/4980
page :1664/4980
page :1665/4980
page :1666/4980
page :1667/4980
page :1668/4980
page :1669/4980
page :16

page :2121/4980
page :2122/4980
page :2123/4980
page :2124/4980
page :2125/4980
page :2126/4980
page :2127/4980
page :2128/4980
page :2129/4980
page :2130/4980
page :2131/4980
page :2132/4980
page :2133/4980
page :2134/4980
page :2135/4980
page :2136/4980
page :2137/4980
page :2138/4980
page :2139/4980
page :2140/4980
page :2141/4980
page :2142/4980
page :2143/4980
page :2144/4980
page :2145/4980
page :2146/4980
page :2147/4980
page :2148/4980
page :2149/4980
page :2150/4980
page :2151/4980
page :2152/4980
page :2153/4980
page :2154/4980
page :2155/4980
page :2156/4980
page :2157/4980
page :2158/4980
page :2159/4980
page :2160/4980
page :2161/4980
page :2162/4980
page :2163/4980
page :2164/4980
page :2165/4980
page :2166/4980
page :2167/4980
page :2168/4980
page :2169/4980
page :2170/4980
page :2171/4980
page :2172/4980
page :2173/4980
page :2174/4980
page :2175/4980
page :2176/4980
page :2177/4980
page :2178/4980
page :2179/4980
page :2180/4980
page :2181/4980
page :2182/4980
page :21

page :2634/4980
page :2635/4980
page :2636/4980
page :2637/4980
page :2638/4980
page :2639/4980
page :2640/4980
page :2641/4980
page :2642/4980
page :2643/4980
page :2644/4980
page :2645/4980
page :2646/4980
page :2647/4980
page :2648/4980
page :2649/4980
page :2650/4980
page :2651/4980
page :2652/4980
page :2653/4980
page :2654/4980
page :2655/4980
page :2656/4980
page :2657/4980
page :2658/4980
page :2659/4980
page :2660/4980
page :2661/4980
page :2662/4980
page :2663/4980
page :2664/4980
page :2665/4980
page :2666/4980
page :2667/4980
page :2668/4980
page :2669/4980
page :2670/4980
page :2671/4980
page :2672/4980
page :2673/4980
page :2674/4980
page :2675/4980
page :2676/4980
page :2677/4980
page :2678/4980
page :2679/4980
page :2680/4980
page :2681/4980
page :2682/4980
page :2683/4980
page :2684/4980
page :2685/4980
page :2686/4980
page :2687/4980
page :2688/4980
page :2689/4980
page :2690/4980
page :2691/4980
page :2692/4980
page :2693/4980
page :2694/4980
page :2695/4980
page :26

page :3147/4980
page :3148/4980
page :3149/4980
page :3150/4980
page :3151/4980
page :3152/4980
page :3153/4980
page :3154/4980
page :3155/4980
page :3156/4980
page :3157/4980
page :3158/4980
page :3159/4980
page :3160/4980
page :3161/4980
page :3162/4980
page :3163/4980
page :3164/4980
page :3165/4980
page :3166/4980
page :3167/4980
page :3168/4980
page :3169/4980
page :3170/4980
page :3171/4980
page :3172/4980
page :3173/4980
page :3174/4980
page :3175/4980
page :3176/4980
page :3177/4980
page :3178/4980
page :3179/4980
page :3180/4980
page :3181/4980
page :3182/4980
page :3183/4980
page :3184/4980
page :3185/4980
page :3186/4980
page :3187/4980
page :3188/4980
page :3189/4980
page :3190/4980
page :3191/4980
page :3192/4980
page :3193/4980
page :3194/4980
page :3195/4980
page :3196/4980
page :3197/4980
page :3198/4980
page :3199/4980
page :3200/4980
page :3201/4980
page :3202/4980
page :3203/4980
page :3204/4980
page :3205/4980
page :3206/4980
page :3207/4980
page :3208/4980
page :32

page :3660/4980
page :3661/4980
page :3662/4980
page :3663/4980
page :3664/4980
page :3665/4980
page :3666/4980
page :3667/4980
page :3668/4980
page :3669/4980
page :3670/4980
page :3671/4980
page :3672/4980
page :3673/4980
page :3674/4980
page :3675/4980
page :3676/4980
page :3677/4980
page :3678/4980
page :3679/4980
page :3680/4980
page :3681/4980
page :3682/4980
page :3683/4980
page :3684/4980
page :3685/4980
page :3686/4980
page :3687/4980
page :3688/4980
page :3689/4980
page :3690/4980
page :3691/4980
page :3692/4980
page :3693/4980
page :3694/4980
page :3695/4980
page :3696/4980
page :3697/4980
page :3698/4980
page :3699/4980
page :3700/4980
page :3701/4980
page :3702/4980
page :3703/4980
page :3704/4980
page :3705/4980
page :3706/4980
page :3707/4980
page :3708/4980
page :3709/4980
page :3710/4980
page :3711/4980
page :3712/4980
page :3713/4980
page :3714/4980
page :3715/4980
page :3716/4980
page :3717/4980
page :3718/4980
page :3719/4980
page :3720/4980
page :3721/4980
page :37

page :4173/4980
page :4174/4980
page :4175/4980
page :4176/4980
page :4177/4980
page :4178/4980
page :4179/4980
page :4180/4980
page :4181/4980
page :4182/4980
page :4183/4980
page :4184/4980
page :4185/4980
page :4186/4980
page :4187/4980
page :4188/4980
page :4189/4980
page :4190/4980
page :4191/4980
page :4192/4980
page :4193/4980
page :4194/4980
page :4195/4980
page :4196/4980
page :4197/4980
page :4198/4980
page :4199/4980
page :4200/4980
page :4201/4980
page :4202/4980
page :4203/4980
page :4204/4980
page :4205/4980
page :4206/4980
page :4207/4980
page :4208/4980
page :4209/4980
page :4210/4980
page :4211/4980
page :4212/4980
page :4213/4980
page :4214/4980
page :4215/4980
page :4216/4980
page :4217/4980
page :4218/4980
page :4219/4980
page :4220/4980
page :4221/4980
page :4222/4980
page :4223/4980
page :4224/4980
page :4225/4980
page :4226/4980
page :4227/4980
page :4228/4980
page :4229/4980
page :4230/4980
page :4231/4980
page :4232/4980
page :4233/4980
page :4234/4980
page :42

page :4686/4980
page :4687/4980
page :4688/4980
page :4689/4980
page :4690/4980
page :4691/4980
page :4692/4980
page :4693/4980
page :4694/4980
page :4695/4980
page :4696/4980
page :4697/4980
page :4698/4980
page :4699/4980
page :4700/4980
page :4701/4980
page :4702/4980
page :4703/4980
page :4704/4980
page :4705/4980
page :4706/4980
page :4707/4980
page :4708/4980
page :4709/4980
page :4710/4980
page :4711/4980
page :4712/4980
page :4713/4980
page :4714/4980
page :4715/4980
page :4716/4980
page :4717/4980
page :4718/4980
page :4719/4980
page :4720/4980
page :4721/4980
page :4722/4980
page :4723/4980
page :4724/4980
page :4725/4980
page :4726/4980
page :4727/4980
page :4728/4980
page :4729/4980
page :4730/4980
page :4731/4980
page :4732/4980
page :4733/4980
page :4734/4980
page :4735/4980
page :4736/4980
page :4737/4980
page :4738/4980
page :4739/4980
page :4740/4980
page :4741/4980
page :4742/4980
page :4743/4980
page :4744/4980
page :4745/4980
page :4746/4980
page :4747/4980
page :47

In [ ]:
df = data_all

In [ ]:
for i in check_list_2:
    print(i)
check_list_2

In [ ]:
#불용어 처리
df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','') 
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')  

In [ ]:
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')

In [ ]:
#불용어 처리
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('.', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.replace(r'/*', '')
df['Content'] = df['Content'].str.replace('◇','')
df['Content'] = df['Content'].str.replace('＜','')
df['Content'] = df['Content'].str.replace('＞','')
df['Content'] = df['Content'].str.replace('△','')
df['Content'] = df['Content'].str.replace('▲','')
df['Content'] = df['Content'].str.replace('=','')
df['Content'] = df['Content'].str.replace('||','')
df['Content'] = df['Content'].str.replace('{','')
df['Content'] = df['Content'].str.replace('}','')
df['Content'] = df['Content'].str.replace('()','')
df['Content'] = df['Content'].str.replace(';','')
df['Content'] = df['Content'].str.replace(')','')
df['Content'] = df['Content'].str.replace('(','')
df['Content'] = df['Content'].str.replace('*','')
df['Content'] = df['Content'].str.replace('＜1-2＞','')
df['Content'] = df['Content'].str.replace('\s{2,}','') 
df['Content'] = df['Content'].str.replace('#\S+','') #delete#tag  
df['Content'] = df['Content'].str.replace('@\S+','') #@xx제거
df['Content'] = df['Content'].str.replace('주요기사.*','') 
df['Content'] = df['Content'].str.replace('©','') 
df['Content'] = df['Content'].str.replace('○',' ')
df['Content'] = df['Content'].str.replace('■','')
df['Content'] = df['Content'].str.replace('●','')
df['Content'] = df['Content'].str.replace('▽','')
df['Content'] = df['Content'].str.replace('《','')
df['Content'] = df['Content'].str.replace('》','')
df['Content'] = df['Content'].str.replace('〃','')
df['Content'] = df['Content'].str.replace(r'ufeff','') 
df['Content'] = df['Content'].str.replace('사진 게티이미지','')

df['Content'] = df['Content'].str.lstrip() #공백제거

In [ ]:
df.insert(5, "URL",href_url, True)

In [ ]:
nows = time.localtime()
times = str(nows.tm_year) + '-' + str(nows.tm_mon) + '-' + str(nows.tm_mday)
name = times + '중앙.xlsx'

writer = pd.ExcelWriter(name, engine='xlsxwriter', options={'strings_to_urls': True})
df.to_excel(writer)
writer.close()